In [2]:
from PIL import Image
import os
import random
import cv2
import numpy as np

def sobrepor_imagens(comb_id,pasta_png, gun_bb, sprite_bb, pasta_fonte, back_spawn_area, pasta_resultados):
    # comb_id -> nome unico 
    # pasta_png -> sprite
    # gun_bb -> gun bounding box
    # sprite_bb
    # pasta_fonte -> b ackground 
    # back_spawn
    # pasta_resultado 
    

    # Certifica-se de que a pasta de resultados existe
    os.makedirs(pasta_resultados, exist_ok=True)

    # Lista as imagens da pasta PNG
    imagens_png = [f for f in os.listdir(pasta_png) if f.endswith('.png')]

    # Lista as imagens da pasta FONTE
    imagens_fonte = [f for f in os.listdir(pasta_fonte) if f.endswith(('.png', '.jpg', '.jpeg'))]

    counter = 0 # for image name
    for img_fonte_nome in imagens_fonte:
        #Carregando background com OpenCV
        caminho_img_fonte = os.path.join(pasta_fonte, img_fonte_nome)
        img_fonte = cv2.imread(caminho_img_fonte)
        img_fonte = cv2.cvtColor(img_fonte, cv2.COLOR_BGR2RGB) 
        img_height, img_width, _ = img_fonte.shape

        gun_coords = [] # coordenadas de cada b.b. de arma
        gun_const = [] # constantes de centro e dimensões de cada b.b. de arma
        sprite_const = [] # constantes de centro e dimensões de cada b.b. de sprite
        nome_resultado = None
        caminho_resultado = None
        label_nome = None
        caminho_label = None

        counter += 1
        sprite_count = 0
        for i,img_png_nome in enumerate(imagens_png): # adicionando um sprite por vez no mesmo background
            if i == len(back_spawn_area): # ajustar loop conforme numero de sprites
                break
            sprite_count += 1
            #Carregando sprite com PIL
            caminho_img_png = os.path.join(pasta_png, img_png_nome)
            img_png = Image.open(caminho_img_png)  
            #convertendo img_fonte para numpy array
            img_fonte = np.array(img_fonte)
            #convertendo para rgb
            #checando se a imagem é rgb
            if img_fonte.shape[2] == 4:
                img_fonte = cv2.cvtColor(img_fonte, cv2.COLOR_RGBA2RGB)
            #variando posição de acordo com a área de spawn de cada sprite
            #print(i)
            variador_x = random.uniform(back_spawn_area[i][0],back_spawn_area[i][2])
            variador_y = random.uniform(back_spawn_area[i][1],back_spawn_area[i][3])
            posicao = (int(variador_x),int(variador_y)) # posição de spawn do sprite
            off_img_x, off_img_y = posicao # offsets da imagem

            #coordenadas finais b.b. do sprite
            x_min_person = sprite_bb[i][0] + off_img_x
            y_min_person = sprite_bb[i][1] + off_img_y
            x_max_person = sprite_bb[i][2] + off_img_x
            y_max_person = sprite_bb[i][3] + off_img_y
            x_center_person =   (x_min_person + x_max_person)/2.0
            y_center_person =   (y_min_person + y_max_person)/2.0
            w_person = x_max_person - x_min_person
            h_person = y_max_person - y_min_person
            sprite_const.append((sprite_count,x_center_person, y_center_person, w_person, h_person))

            #implementação de label automático relativo às posições da arma (colocar sprite como argumento função)
            # para cada sprite com mais de uma arma
            for gun_bb_elem in gun_bb[i]:    
                #print(gun_bb_elem,j)
                sprite_id = sprite_count
                BB_w = gun_bb_elem[2] - gun_bb_elem[0] #bb width
                BB_h = gun_bb_elem[3] - gun_bb_elem[1] #bb height
                sprite_relative_Xoff =  gun_bb_elem[0]
                sprite_relative_Yoff =  gun_bb_elem[1]
                sprite_abs_x_min = sprite_relative_Xoff + off_img_x
                sprite_abs_y_min = sprite_relative_Yoff + off_img_y
                sprite_abs_x_max = sprite_abs_x_min + BB_w
                sprite_abs_y_max = sprite_abs_y_min + BB_h
                sprite_bb_pos_gun = (sprite_abs_x_min, sprite_abs_y_min, sprite_abs_x_max, sprite_abs_y_max) #final absolute bb position
                #coordenadas finais b.b de retangulo da arma
                gun_coords.append(sprite_bb_pos_gun)
                x1, y1, x2, y2 = sprite_bb_pos_gun
                    #bb gun constants
                x_center_gun = (x1+x2)/2.0
                y_center_gun = (y1+y2)/2.0
                w_gun = x2 - x1
                h_gun = y2 - y1
                gun_const.append((sprite_id,x_center_gun, y_center_gun, w_gun, h_gun))
            
            # Cria uma cópia da imagem fonte para preservar a original
            img_fonte = Image.fromarray(img_fonte)
            resultado = img_fonte
            resultado.paste(img_png, posicao, img_png)
            # agora a nova imagem fonte será a imagem com o sprite da vez

            if sprite_count == 1: #se primeira iteração, crie o titulo com o contador de backgrounds
                # Salva a imagem resultante
                nome_resultado = f"{comb_id}_{pasta_fonte.split("/")[-2]}_fundo_{os.path.splitext(img_fonte_nome)[0]}.png"
                caminho_resultado = os.path.join(pasta_resultados, nome_resultado)
                # Salva os labels com mesmo nome no formato txt.
                label_nome = f"{comb_id}_{pasta_fonte.split("/")[-2]}_fundo_{os.path.splitext(img_fonte_nome)[0]}.txt"
                caminho_label = os.path.join(pasta_resultados, label_nome)
            if sprite_count == len(back_spawn_area): # se chegou ao fim do número de sprites, salve a imagem e reinicie o contador
                # Salva a imagem resultante
                resultado.save(caminho_resultado, "PNG")
                sprite_count = 0
        with open(caminho_label, 'w') as label_file:
            # se mesmo sprite_id, grave tudo em rows proximos
            for sprite_info in sprite_const:
                sprite_id, x_center_person, y_center_person, w_person, h_person = sprite_info
                # colocar label de todas armas da vez
                for gun_info in gun_const:
                    gun_id, x_center_gun, y_center_gun, w_gun, h_gun = gun_info
                    if sprite_id == gun_id:
                        label_file.write(f"{0} {x_center_gun/img_width} {y_center_gun/img_height} {(w_gun)/img_width} {(h_gun)/img_height}\n")
                label_file.write(f"{1} {x_center_person/img_width} {(y_center_person/img_height)} {(w_person)/img_width} {(h_person)/img_height}\n")

In [10]:
root = "/Users/mlair/Desktop/Opt Driven/projeto_labels_automaticos/back_e_sprites/combinacoes/"
n = 4
sprites = root + "comb_{}/sprites".format(n)
backgrounds = root + "comb_{}/back_{}".format(n,n+2)
pasta_resultados = root + "comb_{}/montagens".format(n)

#lista de coordenadas
background_spawn_coords = [[31,396,131,405],[193,440,361,466],[550,400,740,420]] # background_spawn_coords = [[],[]] para sprite 1, 2
sprite_gun_bb = [[[78,29,111,46]],[[46,58,177,142]],[[201,107,243,132]]] # para cada sprite [[[]], [[[]]]]
sprite_coords = [[0.32,1.6,112,165],[6,3.8,187,283],[20,13,262,351]] # sprite_coords = [[],[]] para sprite 1, 2. DO MENOR PARA O MAIOR 

#realizar montagem combinatória aditiva
for i in range(1:len(sprite_coords)):
       sobrepor_imagens(i,sprites, sprite_gun_bb[:i+1], sprite_coords[:i+1], backgrounds, background_spawn_coords[:i+1], pasta_resultados)
#sobrepor_imagens(sprites, sprite_gun_bb, sprite_coords, backgrounds, background_spawn_coords, pasta_resultados)
#print(f"Imagens processadas e salvas em '{pasta_resultados}'!")


# realizar combinação
for i in range(len(sprite_coords)):
    for j in range(len(background_spawn_coords)):
        sobrepor_imagens(
            i,
            sprites,
            [sprite_gun_bb[i]],  # lista com elemento único para combinar com background_spawn_coords
            [sprite_coords[i]],  # lista com elemento único para combinar com background_spawn_coords
            backgrounds,
            [background_spawn_coords[j]],  # combina cada elemento de background_spawn_coords com o elemento atual de sprite
            pasta_resultados
        )
